## Scrape the Location Counts

### 1. Use the beautifulsoup library to scrape the data (from the link above) on state names and corresponding number of store locations, for the following chains:
* Starbucks

* Dunkin’ Donuts

In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

In [47]:
# starbucks
response = requests.get("https://worldpopulationreview.com/state-rankings/starbucks-stores-by-state")
starbucks_soup = BeautifulSoup(response.content, "html.parser")
starbucks_tables = starbucks_soup.find_all("table", class_ = "wpr-table")


In [48]:
rows = []
states_list = []

for th in table.find_all("th")[4:]:
    states = th.get_text(strip = True)
    states_list.append(states)

for i, tr in enumerate(table.find_all("tr")[1:]):
    state = states_list[i]

    cells = tr.find_all("td")
    
    stores2023_tag = cells[0].find("a") or cells[0]
    stores2023 = stores2023_tag.get_text(strip = True)

    stores2021_tag = cells[0].find("a") or cells[1]
    stores2021 = stores2021_tag.get_text(strip = True)

    stores2024_tag = cells[0].find("a") or cells[2]
    stores2024 = stores2024_tag.get_text(strip = True)

    # Append this data.
    rows.append({
        "state": state,
        "2023 Stores": stores2023,
        "2021 Stores": stores2021,
        "2024 Stores": stores2024
    })
print(rows)
df = pd.DataFrame(rows)
print(df)

[{'state': 'California', '2023 Stores': '3,080', '2021 Stores': '2,959', '2024 Stores': '3,117'}, {'state': 'Texas', '2023 Stores': '1,346', '2021 Stores': '1,215', '2024 Stores': '1,409'}, {'state': 'Florida', '2023 Stores': '844', '2021 Stores': '786', '2024 Stores': '892'}, {'state': 'Washington', '2023 Stores': '741', '2021 Stores': '739', '2024 Stores': '736'}, {'state': 'New York', '2023 Stores': '692', '2021 Stores': '643', '2024 Stores': '715'}, {'state': 'Illinois', '2023 Stores': '677', '2021 Stores': '621', '2024 Stores': '685'}, {'state': 'Arizona', '2023 Stores': '548', '2021 Stores': '535', '2024 Stores': '579'}, {'state': 'Colorado', '2023 Stores': '495', '2021 Stores': '497', '2024 Stores': '506'}, {'state': 'Ohio', '2023 Stores': '491', '2021 Stores': '446', '2024 Stores': '499'}, {'state': 'Virginia', '2023 Stores': '489', '2021 Stores': '456', '2024 Stores': '510'}, {'state': 'Oregon', '2023 Stores': '359', '2021 Stores': '345', '2024 Stores': ''}, {'state': 'Pennsyl

In [62]:
#dunkin
dunkin_response = requests.get("https://worldpopulationreview.com/state-rankings/dunkin-donuts-by-state")
dunkin_soup = BeautifulSoup(dunkin_response.content, "html.parser")
dunkin_table = dunkin_soup.find("table", class_ = "wpr-table")

In [64]:
rows = []
states_list = []

for th in dunkin_table.find_all("th")[3:]:
    states = th.get_text(strip = True)
    states_list.append(states)

for i, tr in enumerate(dunkin_table.find_all("tr")[1:]):
    state = states_list[i]

    cells = tr.find_all("td")
    
    dunkin_stores2024_tag = cells[0].find("a") or cells[0]
    dunkin_stores2024 = dunkin_stores2024_tag.get_text(strip = True)

    dunkin_stores2023_tag = cells[0].find("a") or cells[1]
    dunkin_stores2023 = dunkin_stores2023_tag.get_text(strip = True)


    # Append this data.
    rows.append({
        "state": state,
        "2024 Stores": dunkin_stores2024,
        "2023 Stores": dunkin_stores2023
    })
print(rows)
df = pd.DataFrame(rows)
print(df)

[{'state': 'New York', '2024 Stores': '1,431', '2023 Stores': '1,414'}, {'state': 'Massachusetts', '2024 Stores': '1,042', '2023 Stores': '1,068'}, {'state': 'Florida', '2024 Stores': '909', '2023 Stores': '883'}, {'state': 'New Jersey', '2024 Stores': '872', '2023 Stores': '866'}, {'state': 'Illinois', '2024 Stores': '711', '2023 Stores': '692'}, {'state': 'Pennsylvania', '2024 Stores': '650', '2023 Stores': '629'}, {'state': 'Connecticut', '2024 Stores': '476', '2023 Stores': '480'}, {'state': 'Maryland', '2024 Stores': '311', '2023 Stores': '303'}, {'state': 'Georgia', '2024 Stores': '287', '2023 Stores': '265'}, {'state': 'Ohio', '2024 Stores': '261', '2023 Stores': '245'}, {'state': 'Virginia', '2024 Stores': '244', '2023 Stores': '230'}, {'state': 'Texas', '2024 Stores': '226', '2023 Stores': '196'}, {'state': 'North Carolina', '2024 Stores': '224', '2023 Stores': '206'}, {'state': 'New Hampshire', '2024 Stores': '216', '2023 Stores': '218'}, {'state': 'Maine', '2024 Stores': '16

### 2. Parse, merge and tidy your data. Think carefully about what the tidy version of this dataset is with multiple years represented on the website.

## Supplemental Data